In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 10:11:44 2022

@author: roy.369
"""


import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import ScalarFormatter
from scipy.interpolate import CubicSpline

In [12]:
def arr_loc(val,X):
   ind = 0
   for i in range(len(X)):
      if(round(X[i],2) == round(val,2)):
         ind = i
         break
   return ind


def power_fit(x,a1,m):
    return a1*x**m
  
def linear_fit(x,a,b):
    return a+b*x
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [13]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1

In [14]:
def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [15]:
def k_point_grid_upper_half_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              K_lab.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


def k_point_grid_full_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_lab.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


In [16]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_full_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [61]:
def plot_func_multiple_beta_hf(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(["red","orange","goldenrod","blue","purple"])
   plt.figure(figsize = (38,30))
   #plt.title(plot_title,fontsize = 60)
   plt.xticks([2,4,6,8,10],fontsize = 100)
   plt.yticks([0.010,0.015,0.020,0.025,0.030],fontsize = 100)
   #plt.xscale('log')
   #plt.yscale('log') 
   #plt.ylabel(plot_y_label,fontsize = 60)
   #plt.xlabel(plot_x_label,fontsize = 60)
   #U_val = np.linspace(1.0,10.0,num=101)
   #Hd_fit = 0.55*np.power(U_val,-2) 
   plt.tick_params(axis='both', which='major', direction='out', pad=20,length = 30, width = 2.5,labelsize = 120)
   plt.tick_params(axis='both', which='minor', direction='out', pad=20,length = 15, width = 1.5,labelsize = 60)
   for jj in range(len(trot)):
       c_1 = next(color_1)
       beta = (float(dtau))*(float(trot[jj]))
       T = round(1/beta,2)
       plt.errorbar(x_val,y_val[:,jj],yerr = y_val_err[:,jj],c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(T))
   #plt.plot(U_val,Hd_fit,linestyle = 'dashed',color=  'black',linewidth = 3)
   #plt.ylim(top=0.1)  
   #plt.grid(True,which='both')
   #plt.axvspan(3.5, 4.5, facecolor='r', alpha=0.2)
   #plt.axvspan(4.0, 6.0, facecolor='b', alpha=0.2)
   plt.legend(loc='best',fontsize = 60,ncol = 5) 
   plt.tight_layout() 
   plt.savefig(plot_name)
   plt.close()



In [62]:
def plot_func_multiple_U_hf(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(['blue','darkgreen','olive','darkgoldenrod','orange','red'])
   plt.figure(figsize = (25,20))
   #plt.title(plot_title,fontsize = 60)
   plt.yticks([0.01,0.02,0.03],fontsize = 80)
   plt.xticks(fontsize = 80)
   #plt.xscale('log')
   #plt.yscale('log') 
   #plt.ylabel(plot_y_label,fontsize = 60)
   #plt.xlabel(plot_x_label,fontsize = 60)
   #U_val = np.linspace(1.0,10.0,num=101)
   U_val =["1.0","4.0","5.0","6.0","8.0","10.0"] 
   #Hd_fit = 0.55*np.power(U_val,-2) 
   for i in range(len(u_val)):
       c_1 = next(color_1)
       plt.errorbar(x_val,y_val[i,:],yerr = y_val_err[i,:],c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"U = %s"%str(U_val[i]))
   #plt.plot(U_val,Hd_fit,linestyle = 'dashed',color=  'black',linewidth = 3)
   #plt.ylim(top=0.1)  
   plt.grid(True,which='both')
   #plt.axvspan(3.5, 4.5, facecolor='r', alpha=0.2)
   #plt.axvspan(4.0, 6.0, facecolor='b', alpha=0.2)
   plt.legend(loc='best',ncol=2,fontsize = 60) 
   plt.tight_layout() 
   plt.savefig(plot_name)
   plt.close()




In [63]:
def equal_time_moment_moment_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm,Graph_dir_hf,N,u,mu,dtau,L):

   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   M2_M2_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   M2_M2_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   M2_M2_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   M2_M2_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Nrm_M2_M2_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Nrm_M2_M2_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))

   Nden = np.zeros((len(u),len(L)))
   Nden_std = np.zeros((len(u),len(L)))
   Ndbn = np.zeros((len(u),len(L)))
   Ndbn_std = np.zeros((len(u),len(L)))
    
   Nhln = np.zeros((len(u),len(L)))
   Nhln_std = np.zeros((len(u),len(L)))
    
   M2 = np.zeros((len(u),len(L)))
   M2_std = np.zeros((len(u),len(L)))

    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       
   T_val = np.zeros(len(L))    
   for k in range(len(L)):
       T_val[k] = 1/(float(dtau)*float(L[k]))
       
   for i in range(len(u)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,mu,dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[i][k] = sys_measure_avg['Density averaged']
           Nden_std[i][k] = sys_measure_std['Density standard deviation']
           Ndbn[i,k] = sys_measure_avg['Doublon number averaged']
           Ndbn_std[i,k] = sys_measure_std['Doublon number standard deviation']
           
           M2[i][k] = Nden[i][k]-2*Ndbn[i][k]
           M2_std[i][k] = np.sqrt(4*Ndbn_std[i][k]*Ndbn_std[i][k]+Nden_std[i][k]*Nden_std[i][k])
           
           Text_dir_m2_m2_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Moment_moment_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])
           Text_dir_m2_m2_conn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Moment_moment_connected_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])

           filename_m2_m2_corr = '%s/Equal_time_Moment_Moment_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_m2_m2_corr_hf,N,u[i],mu,dtau,L[k])
           M2_M2_corr_hf[i,k,:],M2_M2_corr_hf_std[i,k,:] = np.loadtxt(filename_m2_m2_corr,unpack = 'True', usecols = [1,2])

           filename_m2_m2_conn_corr = '%s/Equal_time_Moment_Moment_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_m2_m2_conn_corr_hf,N,u[i],mu,dtau,L[k])
           M2_M2_conn_corr_hf[i,k,:],M2_M2_conn_corr_hf_std[i,k,:] = np.loadtxt(filename_m2_m2_conn_corr,unpack = 'True', usecols = [1,2])


           for r in range(len(r_2_unq)):
               
               Nrm_M2_M2_conn_corr_hf[i,k,r] = (1/(M2[i][k]*M2[i][k]))*(M2_M2_conn_corr_hf[i,k,r])
               Nrm_M2_M2_conn_corr_hf_std[i,k,r] = Nrm_M2_M2_conn_corr_hf[i,k,r]*(M2_M2_conn_corr_hf_std[i,k,r]/M2_M2_conn_corr_hf[i,k,r]+2*M2_std[i][k]/M2[i][k])
               
 
   M2_M2_corr_hf_n0 = np.zeros((len(u),len(L)))
   M2_M2_corr_hf_n1 = np.zeros((len(u),len(L)))
   M2_M2_corr_hf_n2 = np.zeros((len(u),len(L)))
   M2_M2_corr_hf_n3 = np.zeros((len(u),len(L)))

   M2_M2_corr_hf_n0_std = np.zeros((len(u),len(L)))
   M2_M2_corr_hf_n1_std = np.zeros((len(u),len(L)))
   M2_M2_corr_hf_n2_std = np.zeros((len(u),len(L)))
   M2_M2_corr_hf_n3_std = np.zeros((len(u),len(L)))

   M2_M2_conn_corr_hf_n0 = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n0_anl = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n1 = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n2 = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n3 = np.zeros((len(u),len(L)))
    
   M2_M2_conn_corr_hf_n0_std = np.zeros((len(u),len(L)))

   M2_M2_conn_corr_hf_n0_anl_std = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   M2_M2_conn_corr_hf_n3_std = np.zeros((len(u),len(L)))

   Nrm_M2_M2_conn_corr_hf_n0 = np.zeros((len(u),len(L)))
   Nrm_M2_M2_conn_corr_hf_n1 = np.zeros((len(u),len(L)))
   Nrm_M2_M2_conn_corr_hf_n2 = np.zeros((len(u),len(L)))
   Nrm_M2_M2_conn_corr_hf_n3 = np.zeros((len(u),len(L)))
    
   Nrm_M2_M2_conn_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Nrm_M2_M2_conn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Nrm_M2_M2_conn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Nrm_M2_M2_conn_corr_hf_n3_std = np.zeros((len(u),len(L)))


   for i in range(len(u)):
       for k in range(len(L)):

           M2_M2_corr_hf_n0[i][k] = M2_M2_corr_hf[i][k][0]
           M2_M2_corr_hf_n1[i][k] = M2_M2_corr_hf[i][k][1]
           M2_M2_corr_hf_n2[i][k] = M2_M2_corr_hf[i][k][2]
           M2_M2_corr_hf_n3[i][k] = M2_M2_corr_hf[i][k][3]

           M2_M2_corr_hf_n0_std[i][k] = M2_M2_corr_hf_std[i][k][0]
           M2_M2_corr_hf_n1_std[i][k] = M2_M2_corr_hf_std[i][k][1]
           M2_M2_corr_hf_n2_std[i][k] = M2_M2_corr_hf_std[i][k][2]
           M2_M2_corr_hf_n3_std[i][k] = M2_M2_corr_hf_std[i][k][3]

           M2_M2_conn_corr_hf_n0[i][k] = M2_M2_conn_corr_hf[i][k][0]

           M2_M2_conn_corr_hf_n0_anl[i][k] = M2[i][k]*(1-M2[i][k])
           M2_M2_conn_corr_hf_n1[i][k] = M2_M2_conn_corr_hf[i][k][1]
           M2_M2_conn_corr_hf_n2[i][k] = M2_M2_conn_corr_hf[i][k][2]
           M2_M2_conn_corr_hf_n3[i][k] = M2_M2_conn_corr_hf[i][k][3]
           
           M2_M2_conn_corr_hf_n0_std[i][k] = M2_M2_conn_corr_hf_std[i][k][0]
           M2_M2_conn_corr_hf_n0_anl_std[i][k] = np.sqrt(M2_std[i][k]*M2_std[i][k]+4*M2_std[i][k]*M2_std[i][k]*M2[i][k]*M2[i][k])
           M2_M2_conn_corr_hf_n1_std[i][k] = M2_M2_conn_corr_hf_std[i][k][1]
           M2_M2_conn_corr_hf_n2_std[i][k] = M2_M2_conn_corr_hf_std[i][k][2]
           M2_M2_conn_corr_hf_n3_std[i][k] = M2_M2_conn_corr_hf_std[i][k][3]

           Nrm_M2_M2_conn_corr_hf_n0[i][k] = Nrm_M2_M2_conn_corr_hf[i][k][0]
           Nrm_M2_M2_conn_corr_hf_n1[i][k] = Nrm_M2_M2_conn_corr_hf[i][k][1]
           Nrm_M2_M2_conn_corr_hf_n2[i][k] = Nrm_M2_M2_conn_corr_hf[i][k][2]
           Nrm_M2_M2_conn_corr_hf_n3[i][k] = Nrm_M2_M2_conn_corr_hf[i][k][3]
           
           Nrm_M2_M2_conn_corr_hf_n0_std[i][k] = Nrm_M2_M2_conn_corr_hf_std[i][k][0]
           Nrm_M2_M2_conn_corr_hf_n1_std[i][k] = Nrm_M2_M2_conn_corr_hf_std[i][k][1]
           Nrm_M2_M2_conn_corr_hf_n2_std[i][k] = Nrm_M2_M2_conn_corr_hf_std[i][k][2]
           Nrm_M2_M2_conn_corr_hf_n3_std[i][k] = Nrm_M2_M2_conn_corr_hf_std[i][k][3]
           
           
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"


   #=========================================================Comparing_different_T=============================== 
   Pl_name_n1 = "%s/T_comp_Moment_moment_correlation_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n1 = "%s/T_comp_Moment_moment_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_nrm_conn_n1 = "%s/T_comp_Normalized_Moment_moment_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n0 = "%s/T_comp_Moment_moment_correlation_connected_onsite_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n0_anl = "%s/T_comp_analytical_Moment_moment_correlation_connected_onsite_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
        
   Pl_name_m2 = "%s/T_comp_Local_moment_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
  
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,U_val,M2_M2_corr_hf_n1,M2_M2_corr_hf_n1_std,Pl_name_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,M2_M2_conn_corr_hf_n1,M2_M2_conn_corr_hf_n1_std,Pl_name_conn_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,U_val,Nrm_M2_M2_conn_corr_hf_n1,Nrm_M2_M2_conn_corr_hf_n1_std,Pl_name_nrm_conn_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,U_val,M2,M2_std,Pl_name_m2)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,M2_M2_conn_corr_hf_n0,M2_M2_conn_corr_hf_n0_std,Pl_name_conn_n0)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,M2_M2_conn_corr_hf_n0_anl,M2_M2_conn_corr_hf_n0_anl_std,Pl_name_conn_n0_anl)
    
   #Pl_name_n1_u = "%s/U_comp_Moment_moment_correlation_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   #Pl_name_conn_n1_u = "%s/U_comp_Moment_moment_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   #Pl_name_nrm_conn_n1_u = "%s/U_comp_Normalized_Moment_moment_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   
   #Pl_name_m2_u = "%s/U_comp_Local_moment_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
  
   #plot_func_multiple_U_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,T_val,M2_M2_corr_hf_n1,M2_M2_corr_hf_n1_std,Pl_name_n1_u)
   #plot_func_multiple_U_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,T_val,M2_M2_conn_corr_hf_n1,M2_M2_conn_corr_hf_n1_std,Pl_name_conn_n1_u)
   #plot_func_multiple_U_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,T_val,Nrm_M2_M2_conn_corr_hf_n1,Nrm_M2_M2_conn_corr_hf_n1_std,Pl_name_nrm_conn_n1_u)
   #plot_func_multiple_U_hf(N,u,mu,dtau,L,Pl_title_n3,Pl_x_lab,Pl_y_lab,T_val,M2,M2_std,Pl_name_m2_u)
    
   #=================================Fitting data==========================================

   #n_pts = 801
   #U_fit = np.linspace(1.00,10.00,num=n_pts)
   #Cmm1_fit = np.zeros((len(U_fit),len(L)))
   #Nrm_Cmm1_fit = np.zeros((len(U_fit),len(L)))
   
   #U_max_fit = np.zeros(len(L))
   #Nrm_U_max_fit = np.zeros(len(L))
   #U_max_qmc = np.zeros(len(L))

   #Graph_dir_fit = "%s/Data_fit_polynomial"%Graph_dir_hf
   #if not os.path.exists(Graph_dir_fit):
   #   os.makedirs(Graph_dir_fit)


   #for k in range(len(L)):
   #    T_val[k] = 1/(float(dtau)*float(L[k]))
   #    Poly_fit = np.polynomial.polynomial.Polynomial(np.polynomial.polynomial.polyfit(U_val,M2_M2_conn_corr_hf_n1[:,k],10))
   #    Cmm1_fit[:,k] = Poly_fit(U_fit)
   #    Nrm_Poly_fit = np.polynomial.polynomial.Polynomial(np.polynomial.polynomial.polyfit(U_val,Nrm_M2_M2_conn_corr_hf_n1[:,k],10))
   #    Nrm_Cmm1_fit[:,k] = Nrm_Poly_fit(U_fit)    
       
   #    plt.figure(figsize = (20,20))
   #    plt.xticks(fontsize = 40)
   #    plt.yticks(fontsize = 40)
   #    plt.xlabel("U",fontsize = 40)
   #    plt.ylabel(r"$C_{mm}(1)$",fontsize = 40)
   #    plt.errorbar(U_val,Nrm_M2_M2_conn_corr_hf_n1[:,k],yerr = Nrm_M2_M2_conn_corr_hf_n1_std[:,k],marker = 'o',markersize = 15,capsize =5, elinewidth = 3)
   #    plt.plot(U_fit,Nrm_Cmm1_fit[:,k],linewidth = 3,label = "Data_fit")
   #    plt.legend(loc = 'best',fontsize =40)
   #    plt.savefig("%s/Data_fit_normalized_correlator_N_%s_dtau_%s_L_%s_normal_avg.png"%(Graph_dir_fit,N,dtau,L[k]))

   #    plt.figure(figsize = (20,20))
   #    plt.xticks(fontsize = 40)
   #    plt.yticks(fontsize = 40)
   #    plt.xlabel("U",fontsize = 40)
   #    plt.ylabel(r"$C_{mm}(1)$",fontsize = 40)
   #    plt.errorbar(U_val,M2_M2_conn_corr_hf_n1[:,k],yerr = M2_M2_conn_corr_hf_n1_std[:,k],marker = 'o',markersize = 15,capsize =5, elinewidth = 3)
   #    plt.plot(U_fit,Cmm1_fit[:,k],linewidth = 3,label = "Data_fit")
   #    plt.legend(loc = 'best',fontsize =40)
   #    plt.savefig("%s/Data_fit_correlator_N_%s_dtau_%s_L_%s_normal_avg.png"%(Graph_dir_fit,N,dtau,L[k]))

       
   #    U_max_fit[k] = U_fit[np.argmax(Cmm1_fit[:,k])]
   #    Nrm_U_max_fit[k] = U_fit[np.argmax(Nrm_Cmm1_fit[:,k])]
   #    U_max_qmc[k] = U_val[np.argmax(Nrm_M2_M2_conn_corr_hf_n1[:,k])]


   #U_val_LDOS = np.asarray([3.20,3.30,3.40,3.50,3.60,3.70,3.80,3.90,4.00,4.10,4.20,4.300,4.40,4.50,4.700,4.800,4.900,5.00]) #,5.100,5.200,5.500,6.000,6.500])
   #T_c_LDOS =   np.asarray([0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.33,0.33,0.33,0.33,0.345,0.40,0.40,0.417,0.417,0.455,0.50]) #,0.556,0.556,0.667,0.833,1.000])

   #print(T_val,"T")
   #print(U_max_fit,"U_fit")
   #print(U_max_qmc,"U_qmc")

   #plt.figure(figsize = (25,20))
   #plt.xticks(fontsize = 80)
   #plt.yticks(fontsize = 80)
   #plt.plot(T_val,U_max_fit,marker = "o",markersize = 25,label = "Conn")
   #plt.plot(T_val,Nrm_U_max_fit,marker = "o",markersize = 25,label = "Nrm")    
   #plt.plot(T_c_LDOS,U_val_LDOS,marker = "^",markersize = 25) 
   #plt.grid(True,which='both')
   #plt.legend(loc = 'best',fontsize = 50) 
   #plt.savefig("%s/T_max_normalized_moment_correlations.png"%Graph_dir_hf) 
   #coef = np.polyfit(T_val,U_max_fit,1)
   #line_fit = np.poly1d(coef)
   #T_x = np.arange(0,1.1,0.1)
   #U_x = line_fit(T_x)
   #T_w = np.array([0.2848484848484848,0.19878787878787876,0.1533333333333333,0.12424242424242421,0.09999999999999998])
   #U_w = np.array([6.7899159663865545,6.369747899159664,6.30252100840336,6.100840336134454,6])
   #print(U_x[0])
   #plt.figure(figsize = (30,20))
   #plt.xlabel("T",fontsize = 80)
   #plt.ylabel(r"$U_{max}$",fontsize = 80)
   #plt.xticks([0.0,0.25,0.5,0.75,1.0],fontsize = 120)
   #plt.yticks([U_x[0],5.75,6.00,6.25,6.50,6.75,7.00],fontsize = 120)
   #plt.plot(T_val,U_max_fit,marker = 'o',markersize = 55,label = "Data_fit,n = %s"%str(n_pts))
   #plt.plot(T_val,U_max_qmc,marker = 'o',markersize = 15,linewidth = 5,label = "QMC")
   #plt.plot(T_x,U_x,'--k',linewidth = 10)
   #plt.plot(T_w,U_w,marker = 's',markersize = 55,label = "Widom line")
   #plt.legend(loc = 'best',fontsize = 60)
   #plt.axhline(y=U_x[0],linestyle = '--',color = 'b')
   #plt.grid('True')
   #plt.xlim(0,1.01)
   #plt.text(4, 1, 'y=%s'%(U_x[0]))
   #plt.tight_layout()
   #plt.savefig("%s/U_max_Cmm1_N_%s_dtau_%s_n_points_%s.png"%(Graph_dir_hf,N,dtau,str(n_pts)))


   #color_1 = iter(cm.gnuplot(np.linspace(0, 1, len(L)+1)))
   #plt.figure(figsize = (25,20))
   #plt.xticks([2.0,4.0,6.0,8.0,10.0],fontsize = 80)
   #plt.yticks(fontsize = 80)
   #plt.xlabel("U",fontsize = 40)
   #plt.ylabel(r"$C_{mm}(1)$",fontsize = 40)
   #for k in range(len(L)-1,-1,-1):
   #    c_1 = next(color_1)
   #    plt.errorbar(U_val[Uc1_ind:Uc2_ind+1],Mmt_Mmt_corr_hf_n1[Uc1_ind:Uc2_ind+1,k],yerr = Mmt_Mmt_corr_hf_n1_std[Uc1_ind:Uc2_ind+1,k], ls='none',c=c_1,marker = 'o',markersize = 25,capsize =5, elinewidth = 3)
   #    plt.plot(U_fit,Cmm1_fit[:,k],c=c_1,linestyle = '--',linewidth = 3,label = "T=%s"%str(round(T_val[k],2)))
   #plt.legend(loc = 'upper left',fontsize =50)
   #plt.axvspan(3.5,4.2,color='red',alpha=0.5)
  # plt.axvspan(4.5,6.65,color='blue',alpha=0.3)
  # plt.grid('True',which='both')
  # plt.savefig("%s/Data_fit_N_%s_dtau_%s_fitting_compare_normal_avg.png"%(Graph_dir_hf,N,dtau))
       


In [64]:
def main():

    N = "10"
    U = ["1.00","1.50","2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00","7.50","8.00","8.50","9.00","9.50","10.00","10.50","11.00","11.50","12.00"]
    Dtau = "0.05"
    Mu = "0.00"
    Trot = ["20","30","40","50","60"] 
    
    
    Text_dir_main_corr_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations_half_filling'%N
    Text_dir_main_eqm_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_half_filling'%N

    
    Graph_dir_hf = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_half_filling/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged/Moment_moment_correlations'%(N,N,Dtau)
    if not os.path.exists(Graph_dir_hf):
        os.makedirs(Graph_dir_hf)


    equal_time_moment_moment_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm_hf,Graph_dir_hf,N,U,Mu,Dtau,Trot)
    #equal_time_moment_moment_correlation_function_doping_U_scaling(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot[0])



In [65]:
main()

In [ ]:
N_1 = np.asarray([1.2208436724565757, 1.2109181141439205, 1.1910669975186106])
N_2 = np.asarray([1.211864406779661,1.2070217917675543, 1.194915254237288])
N_3 = np.asarray([1.208816338157447,1.2063950498104907,1.2003418289430996])
N_4 = np.asarray([1.197768125317684,1.197768125317684,1.197768125317684])
N_4p5 = np.asarray([1.1973365617433411,1.1973365617433411,1.1973365617433411])
N_5 = np.asarray([1.191283292978208,1.191283292978208,1.1949152542372878])
N_5p5 = np.asarray([
